In [2]:
# Deep Learning with Python Ch8: GAN
#####################################
#  generative adversarial networks  #
#####################################
# GAN generator network
%reset
import numpy as np
import keras
from keras import layers, models

latent_dim = 2
height     = 32
width      = 32
channels   = 3  # RGB image

generator_input = keras.Input(shape=(latent_dim,))

# transform input into a 16x16 128-channel feature map
x = layers.Dense(128*16*16)(generator_input)
# trick 1: add gradient sparsity by using LeakyReLU
x = layers.LeakyReLU()(x) 
x = layers.Reshape((16, 16, 128))(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# upsmaple to 32x32
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# produce a 32x32 3-channel feature map (= shape of a CIFAR10 image)
# trick 2: use tanh instead of sigmoid as the last activation in geneartor 
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)

# initiate the generator model that maps input of shape(latent_dim,) 
# into an image of shape(32, 32, 3)
generator = models.Model(generator_input, x)
generator.summary()

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 32768)             98304     
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_trans

In [4]:
# GAN discriminator network
from keras import optimizers

discriminator_input = layers.Input(shape=(height, width, channels))

x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
# trick 3: use kernel size divisible by stride size to reduce checkerboard artifacts
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)
# trick 4: add stochasticity to induce robustness
x = layers.Dropout(0.7)(x)
# classification layer
x = layers.Dense(1, activation='sigmoid')(x)

# initiate the discriminator model that turns a (32, 32, 3) input into 
# a binary classification (aka real or fake)
discriminator = models.Model(discriminator_input, x)
discriminator.summary()

# use (1) fradient clipping in the optimizer
#     (2) learning-rate decay to stablize training
discriminator_optimizer = optimizers.RMSprop(lr=6e-4, clipvalue=1.0, decay=1e-8)

# configure the discriminator model
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 6, 6, 128)         0         
__________

In [5]:
# adversarial network
# set discriminator weights to non-trainable during GAN
discriminator.trainable = False
gan_input  = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))

# initiate the GAN model
gan = models.Model(gan_input, gan_output)
gan.summary()

# configure the GAN model
gan_optimizer = optimizers.RMSprop(lr=4e-4, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
model_2 (Model)              (None, 32, 32, 3)         5281539   
_________________________________________________________________
model_4 (Model)              (None, 1)                 790913    
Total params: 6,072,452
Trainable params: 5,281,539
Non-trainable params: 790,913
_________________________________________________________________


In [6]:
# implement GAN training
import os
from keras.preprocessing import image

# load CIFAR10 data
(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()
# select frog images (which is class 6)
x_train = x_train[y_train.flatten()==6]
# normalize data
x_train = x_train.reshape((x_train.shape[0],) + (height, width, channels)).astype('float32') / 255

iterations = 1000
batch_size = 20
save_dir   = 'C:\\Users\\Carol\\Desktop\\GAN'

start = 0
for step in range(iterations):
    # sample random points from the latent space
    # trick 5: sample points from the latent space using normal, instead of uniform, distribution
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # decode them to fake images
    generated_img = generator.predict(random_latent_vectors)
    
    # combine fake images with real ones
    stop         = start + batch_size
    real_img     = x_train[start:stop]
    combined_img = np.concatenate([generated_img, real_img])
    
    # assemble labels: fake = 1, real = 0
    labels  = np.concatenate([np.ones((batch_size,1)), np.zeros((batch_size,1))])
    # trick 4: add stochasticity to induce robustness
    labels += 0.05 * np.random.random(labels.shape)
    
    # train the discriminator
    d_loss = discriminator.train_on_batch(combined_img, labels)
    
    # sample random points from the latent space
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    
    # assemble labels: all real (this is a lie!)
    misleading_targets = np.zeros((batch_size,1))
    
    # train the generator via GAN where discriminator weights are frozen
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    # update
    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0
    
    # occasionally saves and plots (every 500 steps here)
    if step % 100 == 0:
        # save model weights
        gan.save_weights('gan.h5')
        # print metrics
        print('discriminator loss:', d_loss)
        print('adversarial loss:', a_loss)
        # save 1 generated image
        img = image.array_to_img(generated_img[0]*255, scale=False)
        img.save(os.path.join(save_dir, 'fake_frog' + str(step) + '.png'))
        # save 1 real image for comparison
        img = image.array_to_img(real_img[0]*255, scale=False)
        img.save(os.path.join(save_dir, 'real_frog' + str(step) + '.png'))

Instructions for updating:
Use tf.cast instead.


C:\Users\Carol\Anaconda3\envs\keras-gpu\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


discriminator loss: 0.7031995
adversarial loss: 0.692527
discriminator loss: 0.7173842
adversarial loss: 0.7668705
discriminator loss: 0.6926194
adversarial loss: 0.7744368
discriminator loss: 0.6904704
adversarial loss: 0.7821866
discriminator loss: 0.6964921
adversarial loss: 0.7693473
discriminator loss: 0.6906345
adversarial loss: 0.8163556
discriminator loss: 0.6857845
adversarial loss: 0.7027354
discriminator loss: 0.7067378
adversarial loss: 0.7681767
discriminator loss: 0.69438946
adversarial loss: 0.743793
discriminator loss: 0.6976177
adversarial loss: 0.73535764
